# [모듈 3.1] 앤드 포인트 배포


---

# 1. 환경 셋업

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%store -r
%store

Stored variables and their in-db values:
challenger_model_artifact                    -> 's3://sagemaker-us-east-1-189546603447/mab-reviews
challenger_train_job                         -> 'blazingtext-220414-0947-008-c4672311'
champion_model_artifact                      -> 's3://sagemaker-us-east-1-189546603447/mab-reviews
champion_train_job                           -> 'blazingtext-2022-04-14-09-41-42-181'
endpoint_name                                -> 'MAP-Inference-Endpoint-2022-04-15-10-03-57'
model_train_inference_docker                 -> '811284229777.dkr.ecr.us-east-1.amazonaws.com/blaz
model_train_inference_docker_uri             -> '811284229777.dkr.ecr.us-east-1.amazonaws.com/blaz


In [7]:
from datetime import datetime
from sagemaker import get_execution_role, session
import boto3

role = get_execution_role()
sm_session = session.Session(boto3.Session())
sm_runtime = boto3.Session().client("sagemaker-runtime")



champion_model_name:  Champion-2022-04-17-07-42-30
challenger_model_name:  Challenger-2022-04-17-07-42-30


# 1. 세이지 메이커 모델 생성

세이지 메이커 모델 생성을 위한 추론용 도컬 이미지, 모델 아티펙트 경로 확인

In [3]:
print("model_train_inference_docker_uri: \n", model_train_inference_docker_uri)
print("\nchampion_model_artifact: \n", champion_model_artifact)
print("\nchallenger_model_artifact: \n", challenger_model_artifact)


model_train_inference_docker_uri: 
 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:1

champion_model_artifact: 
 s3://sagemaker-us-east-1-189546603447/mab-reviews-helpfulness/output/blazingtext-2022-04-17-07-08-39-410/output/model.tar.gz

challenger_model_artifact: 
 s3://sagemaker-us-east-1-189546603447/mab-reviews-helpfulness/output/blazingtext-220414-0947-008-c4672311/output/model.tar.gz


세이지 메이커 모델 생성

In [9]:
champion_model_name = f"Champion-{datetime.now():%Y-%m-%d-%H-%M-%S}"
challenger_model_name = f"Challenger-{datetime.now():%Y-%m-%d-%H-%M-%S}"

print("champion_model_name: ", champion_model_name)
print("challenger_model_name: ", challenger_model_name)


sm_session.create_model(
    name=champion_model_name, 
    role=role, 
    container_defs={"Image": model_train_inference_docker_uri, 
                    "ModelDataUrl": champion_model_artifact}
)

sm_session.create_model(
    name=challenger_model_name, 
    role=role, 
    container_defs={"Image": model_train_inference_docker_uri, 
                    "ModelDataUrl": challenger_model_artifact}
)

champion_model_name:  Champion-2022-04-17-07-47-51
challenger_model_name:  Challenger-2022-04-17-07-47-51


'Challenger-2022-04-17-07-47-51'

# 2. 변형 생성

이제 각각 고유한 모델이 있는 두 가지 변형을 생성합니다(이는 인스턴스 유형 및 개수도 다를 수 있음).

두 변형에 대해 initial_weight를 "1"로 설정했습니다. 즉, 요청의 50%가 Variant1로 이동하고 나머지 50%가 Variant2로 이동됨을 의미합니다. (두 변형에 대한 가중치의 합은 2이고 각 변형에는 1의 가중치 할당이 있습니다. 이는 각 변형이 총 트래픽의 1/2 또는 50%를 수신함을 의미합니다.)

- 변형1 의 이름은 "Champion", 변형2 의 이름은 "Challenger" 입니다.

In [10]:
from sagemaker.session import production_variant

instance_type = "ml.c5.4xlarge"

variant1 = production_variant(
    model_name=champion_model_name,
    instance_type= instance_type,
    initial_instance_count=1,
    variant_name="Champion",
    initial_weight=1,
)
variant2 = production_variant(
    model_name=challenger_model_name,
    instance_type= instance_type,
    initial_instance_count=1,
    variant_name="Challenger",
    initial_weight=1,
)

(variant1, variant2)

({'ModelName': 'Champion-2022-04-17-07-47-51',
  'VariantName': 'Champion',
  'InitialVariantWeight': 1,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.c5.4xlarge'},
 {'ModelName': 'Challenger-2022-04-17-07-47-51',
  'VariantName': 'Challenger',
  'InitialVariantWeight': 1,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.c5.4xlarge'})

# 3. 엔드포인트 배포

두 가지 변형을 SageMaker 싱글 엔드포인트에 배포

In [11]:
%%time 

endpoint_name = f"MAP-Inference-Endpoint-{datetime.now():%Y-%m-%d-%H-%M-%S}"
print(f"EndpointName={endpoint_name}")

sm_session.endpoint_from_production_variants(
    name=endpoint_name, production_variants=[variant1, variant2]
)

EndpointName=MAP-Inference-Endpoint-2022-04-17-07-55-05
-------------!CPU times: user 179 ms, sys: 10.8 ms, total: 190 ms
Wall time: 6min 31s


'MAP-Inference-Endpoint-2022-04-17-07-55-05'

# 4. 앤드포인트 이름 저장

In [12]:
%store endpoint_name

Stored 'endpoint_name' (str)
